In [56]:
import spacy, csv, nltk
from nltk.tag import pos_tag
from collections import Counter
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from fairseq.models.transformer import TransformerModel
from nltk.translate.bleu_score import sentence_bleu

In [2]:
nlp = spacy.load("en_core_web_sm")

In [3]:
# กำหนดชื่อไฟล์ CSV
csv_file_path = 'research_new.csv'

# สร้าง listA และ listB
listA = []
corpus = []

# อ่านไฟล์ CSV และเก็บข้อมูลใน listA และ corpus
with open(csv_file_path, 'r', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader)  # ข้ามหัวข้อ

    for row in csv_reader:
        listA.append(row[0])
        corpus.append(row[1])

# แสดงผลลัพธ์
print("corpus:", corpus)

corpus: ["what is your dad's name?", "what is your teacher's name?", "what is your father's name?", "what is your mother's name?", "what is your grandfather's name?", "what is your grandmother's name?", "what is your elder's name?", "what is your daughter's name?", "what is your grandson's name?", "what is your granddaughter's name?", "what is your uncle's name?", "what is your aunt's name?", "what is your wife's name?", "what is your husband's name?", "what is your child's name?", 'are you deaf?', 'is your father deaf?', 'is your daughter deaf?', 'is your elder deaf?', 'is your younger deaf?', 'is your uncle deaf?', 'is your aunt deaf?', 'is your wife deaf?', 'is your husband deaf?', 'is your child deaf?', 'is your grandchild deaf?', 'is your son a student?', 'who is he?', 'who is she?', 'what is your name again?', 'what is your father name again?', 'what is your mother name again?', 'what is your grandfather name again?', 'what is your grandmother name again?', 'what is your son name

In [57]:
def lemmatize_text_input(text):
    lemmatized_text = [token.lemma_ for token in nlp(text)]
    return " ".join(lemmatized_text)

def lemmatize_corpus(list_corpus):
  lem_corpus = []
  for i in list_corpus:
    lemmatized_corpus = [token.lemma_ for token in nlp(i)]
    cor = " ".join(lemmatized_corpus)
    lem_corpus.append(cor)
  return lem_corpus


def func_tfidf_cosine_lem(query_sentence, corpus_lem, corpus_nl):
    ifidfvectorizer = TfidfVectorizer()
    vectorizer = ifidfvectorizer.fit_transform([query_sentence] + corpus_lem)
    vocab = ifidfvectorizer.get_feature_names_out()
    query_vector = vectorizer.toarray()[0]
    corpus_lem_vectors = vectorizer.toarray()[1:]
    similarities = cosine_similarity([query_vector], corpus_lem_vectors)
    df = pd.DataFrame(corpus_lem_vectors, columns=vocab)
    most_similar_index = similarities.argmax()
    text = "Not found"
    if similarities.argmax() != 0:
        text = corpus_lem[most_similar_index], corpus_nl[most_similar_index]
    return text, df, similarities.max()

In [5]:
lemmatized_query_corpus = lemmatize_corpus(corpus)

In [6]:
def tokenzie_data(data):
    new_data = []
    for i in range(len(data)):
        new_data.append(" ".join([j[1] for j in pos_tag(nltk.word_tokenize(data[i]))]))
    return new_data

In [7]:
new_data = pd.read_csv("new lang pair1.csv")
new_data.drop(columns=["Unnamed: 0"], inplace=True)

In [8]:
keys = list(new_data["ASL"])
values = list(new_data["English"])

In [63]:
def find_most_similar_sentence_new(query_sentence):
    query_sentence = query_sentence.lower()

    query_sentence = [i[1] for i in pos_tag(nltk.word_tokenize(query_sentence))]
    query_sentence = list(map(lambda x: x[1] + str(query_sentence[:x[0]].count(x[1]) + 1) if query_sentence.count(x[1]) > 1 else x[1], enumerate(query_sentence)))
    for i in range(len(query_sentence)):
        if not any(j.isdigit() for j in query_sentence[i]):
            query_sentence[i] = query_sentence[i]+"1"
    query_sentence = " ".join(query_sentence)

    most_similar_sentence_ifidf_cosine_lem, __, __ = func_tfidf_cosine_lem(query_sentence, values, values)

    result = most_similar_sentence_ifidf_cosine_lem

    return query_sentence, result[1]

def find_most_frequent_word(pattern, training_data):
    pattern_words = pattern.split()
    blank_indices = [i for i, word in enumerate(pattern_words) if word == '___']
    
    n = len(pattern_words)
    candidate_lists = {idx: [] for idx in blank_indices}  # Track words for each blank position

    for sentence in training_data:
        words = sentence.split()
        for i in range(len(words) - n + 1):
            ngram = words[i:i + n]
            if all(pw == w or pw == '___' for pw, w in zip(pattern_words, ngram)):
                for idx in blank_indices:
                    candidate_lists[idx].append(ngram[idx])

    # Find most frequent words for each blank
    most_frequent_words = []
    for idx in blank_indices:
        word_counts = Counter(candidate_lists[idx])
        most_common_word = word_counts.most_common(1)[0][0] if word_counts else "_"
        most_frequent_words.append(most_common_word)

    return most_frequent_words

def is_sublist(sublist, main_list):
    if not sublist:
        return True  # Empty list is always a sublist

    i = 0
    for j in range(len(main_list)):
        if main_list[j] == sublist[i]:
            i += 1
            if i == len(sublist):
                return True
    return False

def overlap_merge(lists):
    merged = lists[0][:]  # Start with the first list (make a copy)

    for lst in lists[1:]:
        # Find the maximum overlap between merged and lst
        max_overlap = 0
        for i in range(len(lst)):
            if merged[-(i + 1):] == lst[: i + 1]:  # Compare suffix of merged with prefix of lst
                max_overlap = i + 1  # Store the maximum matched overlap length

        # Append only the non-overlapping part of lst
        merged.extend(lst[max_overlap:])

    return merged

def correct_sentence(sentence):
    # Corrects grammar mistakes in a sentence, including auxiliary verbs, subject-verb agreement, noun singular/plural, and verb conjugation.
    
    # Helper functions
    def correct_auxiliary(token, subject):
        # Corrects auxiliary verbs based on subject agreement.
        if subject.text.lower() == "you":
            return "are"
        elif subject.text.lower() in ["he", "she", "it", "this", "that"]:
            return "is"
        elif subject.text.lower() in ["i"]:
            return "am"
        else:
            return "are"

    def correct_possessive_noun(token, next_token):
        # Corrects noun to possessive form.
        if next_token and next_token.text.lower() == "name" and token.text.lower() == "mother":
            return "mother's"  # Correct for possessive form
        return token.text

    def singularize_noun(noun):
        # Singularizes the noun if it's plural (handles regular pluralization).
        if noun.endswith("ies"):
            return noun[:-3] + "y"  # cities → city
        elif noun.endswith("es") and not noun.endswith("ss"):
            return noun[:-2]  # buses → bus
        elif noun.endswith("s"):
            return noun[:-1]  # dogs → dog
        return noun

    def conjugate_verb(verb, subject):
        # Conjugates verb based on subject.
        if subject.text.lower() in ["he", "she", "it"]:
            if verb == "have":
                return "has"  # Handle special case for "have" → "has"
            return verb + "s" if not verb.endswith("s") else verb  # eat → eats
        elif subject.text.lower() in ["i", "you", "we", "they"]:
            return verb  # Keep base form
        return verb

    # Process the sentence
    doc = nlp(sentence)
    corrected_words = []

    for token in doc:
        text = token.text.lower()
        pos = token.pos_
        dep = token.dep_

        # Identify the subject of the sentence
        subject = next((t for t in doc if t.dep_ == "nsubj"), None)

        # Handle auxiliary verbs (specifically "be")
        if text == "be" and subject:
            corrected_words.append(correct_auxiliary(token, subject))
            continue

        # Handle noun singularization and pluralization
        if pos == "NOUN":
            # Singularize nouns when necessary (e.g., "names" to "name")
            if token.text.lower() == "names":
                corrected_words.append(singularize_noun(token.text))
            else:
                corrected_words.append(correct_possessive_noun(token, doc[token.i + 1] if token.i + 1 < len(doc) else None))
            continue

        # Handle verb conjugation (e.g., "eat" to "eats")
        if pos == "VERB" and subject:
            corrected_words.append(conjugate_verb(text, subject))
            continue

        # Add token as is if no correction is needed
        corrected_words.append(token.text)

    # Check for subject-verb agreement after processing
    final_sentence = " ".join(corrected_words)
    if "mother's" in final_sentence and "are" in final_sentence:
        final_sentence = final_sentence.replace("are", "is")  # Correct "are" to "is" if "mother's" is singular

    return final_sentence

In [41]:
def sign_translator(text_used):
    after_result = find_most_similar_sentence_new(text_used)
    match_word = dict(zip(after_result[0].split(" "), nltk.word_tokenize(text_used)))
    splitted_result_sentence = after_result[1].split(" ")

    result_sentence = [match_word[splitted_result_sentence[i]] if splitted_result_sentence[i] in match_word.keys() else "___" for i in range(len(splitted_result_sentence))]

    subsens = []

    for i in range(len(result_sentence)):
        for j in range(i+1, len(result_sentence)+1):
            temp = []
            temp.append(result_sentence[i])
            temp += result_sentence[i+1:j]
            if temp.count("___") > 1 or temp.count("___") == 0:
                break
            else:
                if len(temp) != 1:
                    subsens.append(temp)
    subsens.append(result_sentence)

    for i in range(len(subsens)):
        if "___" in subsens[i]:
            temp = find_most_frequent_word(" ".join(subsens[i]), lemmatized_query_corpus)
            for j in temp:
                subsens[i][subsens[i].index("___")] = j
            if subsens[i].count("_") != 0:
                subsens[i].remove("_")
        else:
            subsens[i] = subsens[i]

    for i in range(len(subsens)):
        for j in range(len(subsens)):
            if subsens[i] == subsens[j]:
                continue
            elif is_sublist(subsens[j], subsens[i]):
                subsens[j] = subsens[i]
    subsens = list(map(list,set(map(tuple,subsens))))

    res = " ".join(overlap_merge(subsens))
    return correct_sentence(res)

In [60]:
sign_translator("again name your mother ?")

"what is your mother's name again ?"

In [66]:
test_df = pd.read_csv("research_new.csv")
overall_bleu = 0
for j in range(6):
    used_test_df = test_df.sample(100)
    test_x = list(used_test_df["A"])
    test_y = list(used_test_df["B"])
    test_y = [test_y.replace(".", " .") if test_y.endswith(".") else test_y.replace("?", " ?") for test_y in test_y]
    test_df.drop(used_test_df.index, inplace=True)
    result_x = []
    for i in test_x:
        result_x.append(sign_translator(i))
    avg_bleu = 0
    for i in range(len(test_y)):
        avg_bleu += sentence_bleu(test_y, result_x[i])
    print("Set",j+1,"Average BLEU Score :", avg_bleu/len(test_y))
    overall_bleu += avg_bleu/len(test_y)
print()
print("Overall BLEU Score :", overall_bleu/6)

Set 1 Average BLEU Score : 0.8672072045151238
Set 2 Average BLEU Score : 0.8747431246961332
Set 3 Average BLEU Score : 0.8618758114234412
Set 4 Average BLEU Score : 0.8797010898409792
Set 5 Average BLEU Score : 0.8805584645927641
Set 6 Average BLEU Score : 0.8625241605427533

Overall BLEU Score : 0.8711016426018657
